In [2]:
ENDPOINT_URL =
DEPLOYMENT_NAME = 
AZURE_COGNITIVE_SERVICES_RESOURCE =

SyntaxError: invalid syntax (4238553178.py, line 1)

In [3]:
# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)


NameError: name 'os' is not defined

In [18]:
import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-08-01-preview",
)

# 채팅 함수 정의
def chat_with_openai(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content  # 속성 접근 방식으로 수정
        citations = completion.choices[0].message.context['citations'][0]['content']
        # completion.choices[0].message.context
        # {'citations': [{'content': '음식\n손끝이 아래로 향하게 구부린 오른손의 손가락을 왼손의 5지와 1·2·3·4지 사이에 넣고 오른손 등이 스치게 뺀 다음, 손바닥이 위로 향하게 펴서 약간 구부린 왼손 밑에서 같은 모양의 오른손을 두 번 오르내린다.\nhttp://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201224/791296/IMG000357950_700X466.jpg http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151126/217119/IMG000208379_700X466.jpg\nhttp://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201210/784487/MOV000356911_700X466.mp4\n된장찌개', 'title': None, 'url': None, 'filepath': None, 'chunk_id': '0'}], 'intent': '["된장찌개", "What is 된장찌개?", "된장찌개 recipe"]'}
      
        # completion.choices[0].message.context['citations']
        # [{'content': '음식\n손끝이 아래로 향하게 구부린 오른손의 손가락을 왼손의 5지와 1·2·3·4지 사이에 넣고 오른손 등이 스치게 뺀 다음, 손바닥이 위로 향하게 펴서 약간 구부린 왼손 밑에서 같은 모양의 오른손을 두 번 오르내린다.\nhttp://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201224/791296/IMG000357950_700X466.jpg http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151126/217119/IMG000208379_700X466.jpg\nhttp://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201210/784487/MOV000356911_700X466.mp4\n된장찌개', 'title': None, 'url': None, 'filepath': None, 'chunk_id': '0'}]

        # completion.choices[0].message.context['citations'][0]['content']
        # 손끝이 아래로 향하게 구부린 오른손의 손가락을 왼손의 5지와 1·2·3·4지 사이에 넣고 오른손 등이 스치게 뺀 다음, 손바닥이 위로 향하게 펴서 약간 구부린 왼손 밑에서 같은 모양의 오른손을 두 번 오르내린다.
        # http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201224/791296/IMG000357950_700X466.jpg http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151126/217119/IMG000208379_700X466.jpg
        # http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201210/784487/MOV000356911_700X466.mp4

        print(citations) 
        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가
        
        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        # speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, chat_history

    except Exception as e:
        return [("Error", str(e))], chat_history

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("# Azure OpenAI + Cognitive Search + Speech 기반 수화 챗봇")
    chatbot = gr.Chatbot()
    user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
    clear_button = gr.Button("Clear Chat")
    
    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, state])
    clear_button.click(lambda: [], None, chatbot)

# 실행
demo.launch()


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\components\chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


음식
손끝이 아래로 향하게 구부린 오른손의 손가락을 왼손의 5지와 1·2·3·4지 사이에 넣고 오른손 등이 스치게 뺀 다음, 손바닥이 위로 향하게 펴서 약간 구부린 왼손 밑에서 같은 모양의 오른손을 두 번 오르내린다.
http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201224/791296/IMG000357950_700X466.jpg http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151126/217119/IMG000208379_700X466.jpg
http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201210/784487/MOV000356911_700X466.mp4
된장찌개
